In [1]:
#1 
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb


def to_bool(x):
    if "True" in x:
        return True
    else:
        return False


input_path = "/Users/dongjian/data/sales_forcasting/"
df_train = pd.read_csv(
    input_path + 'train.csv',
    usecols=[1, 2, 3, 4, 5],
    dtype={
        'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': bool,
    },
#     converters={
#         'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0,
#         "onpromotion": to_bool
#     },
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

stores = pd.read_csv(
    input_path + 'stores.csv',
    dtype={
        'cluster': 'int32',
        'store_nbr': 'int8',
    })
transactions = pd.read_csv(
    input_path + 'transactions.csv',
    parse_dates=['date'],
    dtype={'transactions': 'int16',
           'store_nbr': 'int8'},
    infer_datetime_format=True)

items = pd.read_csv(
    input_path + "items.csv",
    dtype={
        'item_nbr': 'int32',
        'perishable': 'int8',
        'class': 'int16'
    })

df_test = pd.read_csv(
    input_path + "test.csv",
    usecols=[0, 1, 2, 3, 4],
    dtype={
        'onpromotion': bool
    },
    parse_dates=["date"]  # , date_parser=parser
)
# .set_index(['store_nbr', 'item_nbr', 'date'])
df_train_new = df_train.merge(
    stores, how='left', on='store_nbr').merge(
        transactions, how='left', on=['store_nbr', 'date']).merge(items,how='left',on=['item_nbr'])

df_2017 = df_train_new[df_train.date.isin(
    pd.date_range("2017-05-31", periods=7 * 11))].copy()

In [25]:
df_2017['weekday'] = df_2017.date.apply(lambda x:x.weekday())

In [27]:
df_2017[(df_2017.store_nbr == 2) & (df_2017.item_nbr == 105574) &
        (df_2017.date.isin(pd.date_range(date(2017, 5, 31),
                                         periods=30)))][['weekday','unit_sales']].head(100)

,weekday,unit_sales
50914613,2,6.0
51021044,3,2.0
51127156,4,2.0
51234928,5,5.0
51350284,6,11.0
51467973,0,13.0
51574338,1,15.0
51677991,2,5.0
51780979,3,1.0
51880248,4,10.0


In [28]:
%store df_2017

Stored 'df_2017' (DataFrame)
